## GEDI L4B Example

### Purpose
Demonstrate how to sample the GEDI L4B raster for BioDensity at generated PhoREAL points 

#### Import Packages

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import sliderule
from sliderule import icesat2

#### Initialize SlideRule Python Client

In [ ]:
icesat2.init("slideruleearth.io", verbose=True)

#### Make Processing Request to SlideRule
ATL06-SR request includes the `samples` parameter to specify that GEDI L3 Mean Elevation dataset should be sampled at each generated ATL06 elevation.

In [ ]:
asset = "icesat2"
resource = "ATL03_20220105023009_02111406_005_01.h5"
region = sliderule.toregion('grandmesa.geojson')
parms = { 
    "poly": region['poly'],
    "srt": icesat2.SRT_LAND,
    "len": 100,
    "res": 100,
    "pass_invalid": True, 
    "atl08_class": ["atl08_ground", "atl08_canopy", "atl08_top_of_canopy"],
    "phoreal": {"binsize": 1.0, "geoloc": "center", "use_abs_h": False, "send_waveform": False},
    "samples": {"gedi": {"asset": "gedil4b"}} 
}
gdf = icesat2.atl08p(parms, asset=asset, resources=[resource], keep_id=True)

#### Display GeoDataFrame Columns
Notice the columns that start with "gedi", they are the sampled raster data

In [ ]:
gdf.keys()

#### Print Out File Directory
When a GeoDataFrame includes samples from rasters, each sample value has a file id that is used to look up the file name of the source raster for that value.

In [ ]:
gdf.attrs['file_directory']

#### Filter GeoDataFrame Based on Valid Values

In [ ]:
df = gdf[gdf["gedi.value"] > -9999.0]
df

#### Plot the Different GEDI Values against the SlideRule PhoREAL Values

In [ ]:
# Setup Plot
fig,ax = plt.subplots(num=None, figsize=(10, 8))
fig.set_facecolor('white')
fig.canvas.header_visible = False
ax.set_title("SlideRule vs. GEDI Elevations")
ax.set_xlabel('UTC')
ax.set_ylabel('height (m)')
legend_elements = []

# Plot SlideRule ATL06 Elevations
sc1 = ax.scatter(df.index.values, df["veg_ph_count"].values, c='red', s=2.5)
legend_elements.append(matplotlib.lines.Line2D([0], [0], color='red', lw=6, label='ATL06-SR'))

# Plot GEDI Elevations
sc2 = ax.scatter(df.index.values, df["gedi.value"].values, c='blue', s=2.5)
legend_elements.append(matplotlib.lines.Line2D([0], [0], color='blue', lw=6, label='GEDI'))

# Display Legend
lgd = ax.legend(handles=legend_elements, loc=3, frameon=True)
lgd.get_frame().set_alpha(1.0)
lgd.get_frame().set_edgecolor('white')

# Show Plot
plt.show()